# **Solve a Sudoku with AI** 
***

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.

#### Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
%matplotlib inline

### Rows and Columns
Let's start by labeling rows and columns
* The rows will be labeled by the letters A, B, C, D, E, F, G, H, I
* The columns will be labeled by the numbers 1, 2, 3, 4, 5, 6, 7, 8, 9
* The 3x3 squares won't be labelled, but in the following diagram, they can be seen with alternating colors of grey and white

![alt text](images/sudoku_diagram.png "Sudoku Diagram")

### Boxes, Units and Peers. 
And let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:
* The individual squares at the intersection of rows and columns will be called boxes. These boxes will have labels 'A1', 'A2', ..., 'I9'.
* The complete rows, columns, and 3x3 squares, will be called units. Thus, each unit is a set of 9 boxes, and there are 27 units in total. Also, each box is associated to 3 units.
* For a particular box (such as 'A1'), its peers will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square). Thus, each box has 20 peers.
    
Let's see an example. In the grids below, each set of blue cells represent each of the 3 units of the box at E3. The individual blue cells show the 20 different peers of the box at E3.

![alt text](images/peers.png "Peers of box E3")

Let's get started. First, we'll record rows and columns as strings.

In [2]:
rows = 'ABCDEFGHI'
cols = '123456789'

We'll start by writing a helper function, cross(a, b), which, given two strings — a and b — will return the list formed by all the possible concatenations of a letter s in string a with a letter t in string b.

In [3]:
def cross(a, b):
      return [s+t for s in a for t in b]

Let's create the labels of the boxes

In [4]:
boxes = cross(rows, cols)
boxes[0:20]

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C2']

Let's create the units

In [5]:
row_units = [cross(r, cols) for r in rows]
row_units

# This is the top most row.
# row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']]

In [6]:
column_units = [cross(rows, c) for c in cols]
column_units

# This is the left most column.
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'I9']]

In [7]:
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
square_units

# This is the top left square.
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

[['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
 ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6'],
 ['A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3'],
 ['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'],
 ['D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3'],
 ['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6'],
 ['G7', 'G8', 'G9', 'H7', 'H8', 'H9', 'I7', 'I8', 'I9']]

In [8]:
unitlist = row_units + column_units + square_units
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

Let's create a dictionary containing a list of the 3 units associated to each box

In [9]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)

# These are the 3 units associated to the A1 box
units['A1']

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]

Let's create a dictionary containing a list of the 20 peers associated to each box

In [10]:
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

# These are the 20 peers of the A1 box
peers['A1']

{'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'E1',
 'F1',
 'G1',
 'H1',
 'I1'}

### Encoding the Board

Now, in order to implement an agent, let's start by coding the board in Python. Then, we'll code the necessary functions to solve the Sudoku. We'll record the puzzles in two ways — as a string and as a dictionary.

The string will consist of a concatenation of all the readings of the digits in the rows, taking the rows from top to bottom. If the puzzle is not solved, we can use a . as a placeholder for an empty box.

For example, an unsolved sudoku will be written as: ..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..

And the solved sudoku will be recorded as: 483921657967345821251876493548132976729564138136798245372689514814253769695417382

We'll implement the dictionary as follows. The keys will be strings corresponding to the boxes — namely, 'A1', 'A2', ..., 'I9'. The values will either be the digit in each box (if there is one) or a '123456789' (if not).

This function converts the string representation of a sudoku into a dictionary form with the value in corresponding box, e.g. '8', or '123456789' if it is empty

In [11]:
def grid_values(string_representation):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in string_representation:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

Test the grid_values() function

In [12]:
test_string_representation = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

test1 = grid_values(test_string_representation)
test1

{'A1': '123456789',
 'A2': '123456789',
 'A3': '3',
 'A4': '123456789',
 'A5': '2',
 'A6': '123456789',
 'A7': '6',
 'A8': '123456789',
 'A9': '123456789',
 'B1': '9',
 'B2': '123456789',
 'B3': '123456789',
 'B4': '3',
 'B5': '123456789',
 'B6': '5',
 'B7': '123456789',
 'B8': '123456789',
 'B9': '1',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '1',
 'C4': '8',
 'C5': '123456789',
 'C6': '6',
 'C7': '4',
 'C8': '123456789',
 'C9': '123456789',
 'D1': '123456789',
 'D2': '123456789',
 'D3': '8',
 'D4': '1',
 'D5': '123456789',
 'D6': '2',
 'D7': '9',
 'D8': '123456789',
 'D9': '123456789',
 'E1': '7',
 'E2': '123456789',
 'E3': '123456789',
 'E4': '123456789',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '123456789',
 'E9': '8',
 'F1': '123456789',
 'F2': '123456789',
 'F3': '6',
 'F4': '7',
 'F5': '123456789',
 'F6': '8',
 'F7': '2',
 'F8': '123456789',
 'F9': '123456789',
 'G1': '123456789',
 'G2': '123456789',
 'G3': '2',
 'G4': '6',
 'G5': '123456789',
 'G6

This function takes the dictionary representation of a sudoku and displays it as a 2D grid

In [13]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

Test the display() function

In [14]:
display(test1)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

### Solving the Sudoku

#### Strategy 1: Elimination
If a box has a value assigned, then none of the peers of this box can have this value.

We can take one pass, go over every box that has a value, and eliminate the values that can't appear on the box, based on its peers. Once we do so, the board looks like this (for clarity, the original filled-in boxes are in bold lettering)

![alt text](images/elimination.png "Possible values by elimination")

This function iterates over all the boxes in the sudoku that only have one value assigned to them, and removes this value from every one of its peers.

In [15]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

Test the eliminate() function and display resulting 2D grid after one iteration

In [16]:
test1 = eliminate(test1)
display(test1)

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


#### Strategy 2: Only Choice
If there is only one box in a unit which would allow a certain digit, then that box must be assigned that digit.

For example, in the unit highlighted in red, there seems to be only one box which would allow a value of 1. Since each digit must appear somewhere in the unit, we conclude that the top right box must contain the digit 1

![alt text](images/only-choice.png "Only Choice")

The function goes through all the units, and if there is a unit with a digit that only fits in one possible box, it will assign that digit to that box

In [17]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            # dplaces is a list of boxes from the current unit with 'digit' as one of its possible values
            dplaces = [box for box in unit if digit in values[box]]
            # if only one box from the current unit with 'digit' as one of its possible values, assign it the value of 'digit'
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

Test the only_choice() function and display resulting 2D grid after one iteration

In [18]:
test1 = only_choice(test1)
display(test1)

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8    278    1   
  2    257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17    6     9   
  6     9     5   |  4     1     7   |  3     8     2   


The function reduce_puzzle() receives as input an unsolved sudoku and applies eliminate() and only_choice() repeatedly in an attempt to solve it

In [19]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

Test the reduce_puzzle() function and display resulting 2D grid after the sudoku has been solved

In [20]:
test1 = reduce_puzzle(test1)
display(test1)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


The algorithm was able to solve the sudoku! Let's see if the algorithm will work on a harder sudoku.

In [21]:
test2_string_representation = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

test2 = grid_values(test2_string_representation)
test2 = reduce_puzzle(test2)
display(test2)

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


The algorithm didn't solve that harder sudoku. It seemed to reduce every box to a number of possibilites, but it won't go farther than that. Let's think of other ways to improve it.

#### Strategy 3: Search
Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively.

For example, the box 'G2' has two possibilities: 8 and 9. We fill it in with a 8 and try to solve our puzzle. If we can't solve it, we'll try with a 9. We are thus creating a tree of possibilities. We could pick any box, but we picked G2 because it has the fewest numbers to try out, so it will be faster to solve. The following diagram shows how a depth-first search works.

![alt text](images/search.png "Depth-first Search")

This function creates a tree of possibilities and traverses it using Depth-First Search until it finds a solution for the sudoku puzzle

In [22]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[box]) == 1 for box in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,box = min((len(values[box]), box) for box in boxes if len(values[box]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[box]:
        new_sudoku = values.copy()
        new_sudoku[box] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

Test the search() function and display the resulting 2D grid after the sudoku has been solved

In [23]:
test2 = search(test2)
display(test2)

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


Our algorithm can now solve harder Sudoku puzzles! But let's extend our solution to include more heuristics...

### Solving a Diagonal Sudoku
A diagonal sudoku is like a regular sudoku, except that among the two main diagonals, the numbers 1 to 9 should all appear exactly once. This means that the boxes on the two diagonals now have 4 units instead of 3, and 26 peers instead of 20. Finally, the center box E5 now has 5 units and 32 peers.

![alt text](images/diagonal-sudoku.png "Diagonal Sudoku")

Let's create the 2 additional units

In [24]:
diagonal_units = [
        ['A1','B2','C3','D4','E5','F6','G7','H8','I9'],
        ['I1','H2','G3','F4','E5','D6','C7','B8','A9']
    ]
diagonal_units

[['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
 ['I1', 'H2', 'G3', 'F4', 'E5', 'D6', 'C7', 'B8', 'A9']]

In [25]:
unitlist_diagonal = unitlist + diagonal_units
unitlist_diagonal

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

Let's modify the dictionary containing a list of the units associated to each box to include the new additional diagonal units

In [26]:
units_diagonal = dict((s, [u for u in unitlist_diagonal if s in u]) for s in boxes)

# These are the 5 units associated to the E5 box
units_diagonal['E5']

[['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'],
 ['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
 ['I1', 'H2', 'G3', 'F4', 'E5', 'D6', 'C7', 'B8', 'A9']]

Let's modify the dictionary containing a list of the peers associated to each box to include the new additional diagonal units

In [27]:
peers_diagonal = dict((s, set(sum(units_diagonal[s],[]))-set([s])) for s in boxes)

# These are the 32 peers of the E5 box
peers_diagonal['E5']

{'A1',
 'A5',
 'A9',
 'B2',
 'B5',
 'B8',
 'C3',
 'C5',
 'C7',
 'D4',
 'D5',
 'D6',
 'E1',
 'E2',
 'E3',
 'E4',
 'E6',
 'E7',
 'E8',
 'E9',
 'F4',
 'F5',
 'F6',
 'G3',
 'G5',
 'G7',
 'H2',
 'H5',
 'H8',
 'I1',
 'I5',
 'I9'}

Add is_diagonal argument to the strategies functions and default with False

In [28]:
def eliminate(values, is_diagonal=False):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    if is_diagonal:
        peers_f = peers_diagonal
    else:
        peers_f = peers
        
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers_f[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [29]:
def only_choice(values, is_diagonal=False):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    if is_diagonal:
        unitlist_f = unitlist_diagonal
    else:
        unitlist_f = unitlist

    for unit in unitlist_f:
        for digit in '123456789':
            # dplaces is a list of boxes from the current unit with 'digit' as one of its possible values
            dplaces = [box for box in unit if digit in values[box]]
            # if only one box from the current unit with 'digit' as one of its possible values, assign it the value of 'digit'
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [30]:
def reduce_puzzle(values, is_diagonal=False):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values, is_diagonal)
        # Use the Only Choice Strategy
        values = only_choice(values, is_diagonal)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [31]:
def search(values, is_diagonal=False):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values, is_diagonal)
    if values is False:
        return False ## Failed earlier
    if all(len(values[box]) == 1 for box in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,box = min((len(values[box]), box) for box in boxes if len(values[box]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[box]:
        new_sudoku = values.copy()
        new_sudoku[box] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

Let's see if the algorithm now works on a diagonal sudoku.

In [32]:
test3_string_representation = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

test3 = grid_values(test3_string_representation)
test3 = search(test3, is_diagonal=True)
display(test3)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


Our algorithm can now solve Diagonal Sudoku puzzles!

#### Optional Technique: Naked Twins
The naked twins technique is the following. Consider the following puzzle, and look at the two highlighted boxes, 'F3' and 'I3'.

As we can see, both belong to the same unit, and both permit the values of 2 and 3. Now, we don't know which one has a 2 and which one has a 3, but we know one thing for sure — the values 2 and 3 are locked in those two boxes, so no other box in their same unit (the third column) can contain the values 2 or 3.

Thus, we go over all the boxes in their same unit, and remove the values 2 and 3 from their possible values.

![alt text](images/naked-twins.png "Naked Twins Technique")

This function finds all the instances of naked twins in all the units and eliminates the naked twins as possibilities for their peers in the same unit

In [33]:
def naked_twins(values, is_diagonal=False):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    if is_diagonal:
        unitlist_f = unitlist_diagonal
    else:
        unitlist_f = unitlist
        
    # Find all instances of naked twins
    for unit in unitlist_f:
        # Find boxes in the unit with two possible values
        boxes_with_two_possible_values = [box for box in unit if len(values[box])==2]
        for b in boxes_with_two_possible_values:
            # Find if each of those boxes have a twin
            b_twin = [box for box in boxes_with_two_possible_values if values[box] == values[b] and box != b]
            if b_twin:
                # Eliminate the naked twins as possibilities for their peers
                digits_twins = values[b]
                for digit in digits_twins:
                    for peer in unit:
                        if peer not in [b,b_twin[0]]:
                            values[peer] = values[peer].replace(digit,'')
    return values

Test that the naked_twins() function works in the example unit of the previous diagram

In [34]:
test4_string_representation = '1.4.9..68956.18.34..84.695151.....868..6...1264..8..97781923645495.6.823.6.854179'

test4 = grid_values(test4_string_representation)
test4 = eliminate(test4)
print('Pre Naked-Twins State:\n')
display(test4)

Pre Naked-Twins State:

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


In [35]:
test4 = naked_twins(test4)
print('Post Naked-Twins State:\n')
display(test4)

Post Naked-Twins State:

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    3    79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


The naked_twins() function effectively eliminated '2' and '3' as possible values for the 'D3' and 'E3' boxes